In [1]:
import os
import fitz  # PyMuPDF
from docx import Document
from tqdm import tqdm
import re

# === PARAMETRI ===
CARTELLA_DOCUMENTI = "./documents_for_training"

# === FUNZIONE: Pulizia testo ===
def pulisci_testo(testo):
    testo = re.sub(r'\n{2,}', '\n', testo)  # Rimuove righe vuote multiple
    testo = re.sub(r'[ \t]+', ' ', testo)   # Spazi o tab ripetuti
    testo = testo.strip()
    return testo

# === FUNZIONE: Parsing PDF ===
def estrai_testo_pdf(path):
    testo = ""
    try:
        with fitz.open(path) as doc:
            for pagina in doc:
                testo += pagina.get_text()
    except Exception as e:
        print(f"Errore PDF {path}: {e}")
    return pulisci_testo(testo)

# === FUNZIONE: Parsing DOCX ===
def estrai_testo_docx(path):
    testo = ""
    try:
        doc = Document(path)
        for par in doc.paragraphs:
            testo += par.text + "\n"
    except Exception as e:
        print(f"Errore DOCX {path}: {e}")
    return pulisci_testo(testo)

# === MAIN: Scansione della cartella ===
documenti = {}  # filename → testo

for root, _, files in os.walk(CARTELLA_DOCUMENTI):
    for nome_file in tqdm(files, desc="Parsing documenti"):
        percorso = os.path.join(root, nome_file)
        estensione = nome_file.lower().split(".")[-1]

        if estensione == "pdf":
            testo = estrai_testo_pdf(percorso)
        elif estensione == "docx":
            testo = estrai_testo_docx(percorso)
        else:
            continue  # ignora altri formati

        if len(testo) > 100:  # ignora file troppo corti
            documenti[nome_file] = testo

print(f"Totale documenti estratti: {len(documenti)}")

# === OPTIONAL: Salva in file json per debug ===
import json
with open("documenti_puliti.json", "w", encoding="utf-8") as f:
    json.dump(documenti, f, indent=2, ensure_ascii=False)


Parsing documenti: 100%|██████████| 11/11 [00:01<00:00,  6.53it/s]

Totale documenti estratti: 8


In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import datetime

SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']
SERVICE_ACCOUNT_FILE = 'credentials_google.json'  # tuo file di credenziali

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

service = build('calendar', 'v3', credentials=credentials)

now = datetime.datetime.utcnow().isoformat() + 'Z'  # tempo UTC ISO
events_result = service.events().list(calendarId='primary', timeMin=now,
                                      maxResults=50, singleEvents=True,
                                      orderBy='startTime').execute()
events = events_result.get('items', [])

testo_eventi = ""
for event in events:
    start = event['start'].get('dateTime', event['start'].get('date'))
    summary = event.get('summary', 'No title')
    descr = event.get('description', '')
    testo_eventi += f"{start} - {summary}\n{descr}\n\n"

print(testo_eventi)


MalformedError: Service account info was not in the expected format, missing fields client_email, token_uri.